# Daily Top 10 Topic
# by PARK-SI HYUNG. 2019-02-08
#    

---

![JPEG](http://www.thesmania.com/wp-content/uploads/2016/02/good-topic.jpg)

- 다음 뉴스를 Topic 별로 크롤링합니다
- 전날 06시 ~ 24시, 당일 00시 ~ 05시 까지의 뉴스를 수집합니다
- [경향포토]같은 불필요한 단어들은 제목에서 전처리해줍니다
- 뉴스에서 언급된 단어 중 순위별로 10개를 보여드립니다

#   


In [1]:
import time
import requests
from bs4 import BeautifulSoup
import lxml
from collections import Counter
from datetime import date, timedelta
from joblib import Parallel, delayed

# 자동으로 어제와 오늘 날짜를 구합니다
#   

In [2]:
# 어제 날짜와 오늘 날짜 구하기

date_list = []

def search_date():
    
    today = int(date.today().strftime('%Y%m%d'))
    yesterday = date.today() - timedelta(1)
    yesterday = int(yesterday.strftime('%Y%m%d'))
    
    date_list.append(yesterday)
    date_list.append(today)
    
search_date()
date_list

[20190303, 20190304]

In [3]:
# 우리가 원하는 8개의 토픽들
topics = ['society', 'politics', 'economic', 'foreign', 'culture',
          'entertain', 'sports', 'digital']

# 분석에 필요한 뉴스 페이지 검색

- 우리가 필요한 데이터는 전날 오전06시 ~ 오후24시, 당일 오전 06시 전까지의 데이터입니다
- 오전 7시에 배포한다고 가정했을 때, 1시간정도는 분석하기 위해서 오전 6시까지 모읍니다

#   

In [4]:
def date_crawl(page):
    
    try:
        yesterday = []
        today = []
    
        url = 'http://media.daum.net/breakingnews/{}?page={}&regDate={}' \
        .format(topic, page, date)   
        html = requests.get(url)
        soup = BeautifulSoup(html.text, 'html.parser')
        news_time = soup.find_all(class_ = 'info_time')
    
    
    
        news_time = news_time[0].text
        news_time = int(news_time[:2])
        
        # 전날 새벽6시 ~ 오후 24시까지의 데이터를 원함
        if(i==0 and news_time > 5):
            yesterday.append(page)
        
        # 당일 새벽6시 전까지의 데이터를 원함
        if(i==1 and news_time < 6):
            today.append(page)
        
        time.sleep(0.01)
    
        if(i==0):
            return yesterday
        if(i==1):
            return today
    except:
        pass

In [5]:
# 1000페이지까지 크롤링하면서 우리가 원하는 시간대의 페이지를 얻는다
# 8개 토픽들의 어제 06시 ~ 오후 24시, 오늘 06시 전까지의 페이지를 구한다
start_time = time.time()
total_search_page = []

# 8개 토픽 모두 크롤링
for topic in topics:
    
    
    
    print('{} 분석에 필요한 페이지를 추출합니다'.format(topic))
    
    # 어제, 오늘 순으로 크롤링
    for i, date in enumerate(date_list):
        
        
        page_info = Parallel(n_jobs=-1)(delayed(date_crawl)(page) for page in range(1,601))
        page_info = [x for x in page_info if x]
        page_info = sum(page_info, [])
        
        if i == 0:
            total_search_page.append(page_info)
        elif i == 1:
            total_search_page.append(page_info)
            
        time.sleep(0.01)
        
        
print('\n') 
print("총 걸린시간 : {}분".format(round((time.time() - start_time)/60, 1)))

society 분석에 필요한 페이지를 추출합니다
politics 분석에 필요한 페이지를 추출합니다
economic 분석에 필요한 페이지를 추출합니다
foreign 분석에 필요한 페이지를 추출합니다
culture 분석에 필요한 페이지를 추출합니다
entertain 분석에 필요한 페이지를 추출합니다
sports 분석에 필요한 페이지를 추출합니다
digital 분석에 필요한 페이지를 추출합니다


총 걸린시간 : 6.9분


In [18]:
# 1. 필요한 페이지를 추출함
# 2. 구한 페이지를 토대로 제목과 URL을 리스트에 따로 저장해둠
# 3. 제목을 단어로 잘라내 top10 keyword를 구함
# 4. keyword가 들어있는 제목과 url을 찾아서 DataFrame을 생성
# 5. 크롤링 한번으로 모두 수집 가능

# 제목 다음에 url을 저장한다

# 제목에서 KEYWORD를 추출합니다

- [  혹은  ]가 들어간 제목은 제외합니다 ([경향포토]같은))
- 한자리 단어도 제외합니다.
- 크롤링 할때마다 지연시간을 줘서 디도스 의심을 제거합니다(!?)

#   

In [19]:
# 크롤링 한번으로 키워드, 제목, url 모두 구하자
# '추천연재'가 자동으로 크롤링되서 혼란을 주기때문에 맨마지막 2개만 뺀다

# 크롤링 한번으로 키워드, 제목, url 모두 구하자
# '추천연재'가 자동으로 크롤링되서 혼란을 주기때문에 맨마지막 2개만 뺀다

def crawling_keyword_title_url(page):
    
    title_to_word = []
    title_and_address = []
    
    
    # 페이지 지정
    url = 'http://media.daum.net/breakingnews/{}?page={}&regDate={}' \
    .format(topic, page, date)   
    
    html = requests.get(url)
    soup = BeautifulSoup(html.text, 'html.parser')
    
    if(len(soup.select('div.cont_thumb strong a')[0:15]) == 2):
            return
    
    # 제목에서 키워드을 검색
    for title in soup.select('div.cont_thumb strong a')[0:15]:
        
        
        
        # 키워드를검색하기 위해 페이지에서 제목부터 추출
        title = str(title)
        # 제목은 <a> </a>사이에 있다
        # index로 검색
        title_first_index = title.index('>')
        title_last_index = title.index('</a>')
        # 인덱스 앞뒤로 제목을 추출
        title_name = title[title_first_index+1: title_last_index]
        title_name = title_name.replace("'",'').replace('"','').replace('..','').replace('?','').replace('!','').replace(',','').replace('·','').replace('↑','')
        
        # 제목을 분석해 키워드를 분석합니다
        words = title_name.split()
        title_to_word.append(words)
            
        # 제목에 []가 들어간 것은 제목에서 빼준다
        # [르포] 혹은 [경향포토]
        if '[' and ']' in title_name:
            first_index = title_name.find('[')
            last_index = title_name.find(']')
            
            if(first_index == 0):
                title_name = title_name[last_index+1:]
                    
            else:
                title_name = title_name[:first_index]
                    
        
        # 기사내용을 크롤링하기 위해서 url 추출
        url_first_index = title.index('http')
        url_last_index = title.index('>')
        address = title[url_first_index:url_last_index-1]
        
        # 마지막에 DataFrame 생성을 위해 기사제목과 url은 따로 저장
        # 제목, url 순으로 list에 저장
        title_and_address.append(title_name)
        title_and_address.append(address)
        
    return title_to_word ,title_and_address
        
    time.sleep(0.1)
    

In [20]:
# 추출한 페이지에서 뉴스 제목을 검색
# 제목을 추출해서 단어별로 나열합니다
title_address_list = []
total_top10_keyword = []
start_page = 0
page_list = []
title_address = []
start_time = time.time()

for topic in topics:
    
    print('{} 분석에 필요한 제목을 분석합니다'.format(topic))
    
    topic_keyword = []
    
    # 디도스를 피하기 위해 time sleep
    time.sleep(3)
    
    # i는 전날이면 0, 오늘이면 1
    for i, date in enumerate(date_list):
        
        
        title_info = Parallel(n_jobs=-1)(delayed(crawling_keyword_title_url)(page) for page in total_search_page[start_page+i][:])
        title_info = [x for x in title_info if x]
        
        for p in title_info:
            
            topic_keyword.append(p[0])
            title_address_list.append(p[1])
            
    start_page+=1
    
    # 토픽별로 리스트에 통합시킨다
    topic_keyword = sum(topic_keyword, [])
    total_top10_keyword.append(topic_keyword)
    
title_address_list = sum(title_address_list, [])

print('\n')
print("총 걸린시간 : {}분".format(round((time.time() - start_time)/60, 1)))

society 분석에 필요한 제목을 분석합니다
politics 분석에 필요한 제목을 분석합니다
economic 분석에 필요한 제목을 분석합니다
foreign 분석에 필요한 제목을 분석합니다
culture 분석에 필요한 제목을 분석합니다
entertain 분석에 필요한 제목을 분석합니다
sports 분석에 필요한 제목을 분석합니다
digital 분석에 필요한 제목을 분석합니다


총 걸린시간 : 1.2분


In [21]:
keyword_ranking = []

for i in range(len(total_top10_keyword)):
    # 1차원 리스트로 합쳐준다
    keyword_ranking.append(sum(total_top10_keyword[i], []))

# 영어로 된 제목을 처리해줍니다
#### foreign, sports 쪽은 영어가 압도적으로 많습니다. 한국어 제목만을 추출해줍니다
#   

In [22]:
# sports와 foreign 분야는 영어 제목이 많아 제목에서 영어를 빼줍니다
# 한글로 된 제목만 취급합니다
import re

no_english_foreign_title = []
no_english_sports_title = []

for word in keyword_ranking[3]:
    text = re.sub('[a-zA-Z]','',word).strip()
    if(text!=''):
        no_english_foreign_title.append(text)

for word in keyword_ranking[6]:
    text = re.sub('[a-zA-Z]','',word).strip()
    if(text!=''):
        no_english_sports_title.append(text)
        
keyword_ranking[3] = no_english_foreign_title
keyword_ranking[6] = no_english_sports_title

# Topic별로 1~10위 까지의 Keyword를 보여줍니다
- 1자리 단어, '[' 혹은 ']'가 들어간 단어는 제외합니다
- 대명사같이 불필요한 단어는 필터링 해 줍니다

#   


In [23]:
keyword_filter = [['관련','지지','호소하는','하는','후보','회장','인사하는','개최', '대표','장관', '의혹', '지원', '발표', '추진', '서울'],
                  ['뒷받침','발언하는', '오늘', '정상','대표','신임'],
                  ['투자','시장','5%','이벤트','기념','기업','오른','최대','영업이익','이평선','상승세','지난해', '영업익', '출시', '대비', '상승', '전일', '추가', '개최', '실시간', '분석', '매수', '종목알파고', '외국인', '수급과', '확률은', '규모', '증가', '결정', '확대', '공개', '실적', '대표', '선임', '이상', '올해', '하락', '지원', '주당', '작년', '회장', '2019','감소','만에','위한','전달','마감','강화','발표','사장','대한민국','하락한'],
                  ['도착','--','만에','()'],
                  ['진행','위한','발간','출시','이데일리', '공개', '나쁨', '개최', '왔어요', '대표', '서울'],
                  ['닿다', '눈빛','진심이','공개', '데뷔', '3월', '소녀', '1위', '이달의', '4월', '의혹', '확정', '출시', '매력', '남편'],
                  ['밝히는', '각오','선수','선임','도전'],
                  ['사업', '추진', '분야', '모집', '개최','글로벌','미래']]

In [24]:
# 키워드 필터링

filter_index = 0

for keyword in keyword_ranking:
    keyword_ranking[filter_index] = [x for x in keyword if not x in keyword_filter[filter_index]]
    filter_index+=1

In [25]:
# 리스트에서 1자리 단어는 빼버림
# []가 앞뒤로 시작하는 단어도 빼버림 ([경향포토]같은)


total_keyword_ranking10 = []

for i in range(len(keyword_ranking)):
    # 1위부터 보여줍니다
    ranking = 1
    
    replace = [x for x in keyword_ranking[i] if not len(x)==1 and not "[" in x and not "]" in x]
    
    print('{}를 분석합니다\n'.format(topics[i]))
    
    temp_word = []
    
    for word, count in Counter(replace).most_common(20):
        
        # Top 20 topic
        print('가장 많이 등장한 Keyword {}위: {} / 등장 횟수 : {}'.format(ranking, word, count+1))
        temp_word.append(word)
        
        ranking+=1
        
    total_keyword_ranking10.append(temp_word[0:10])
    print('\n')
    

society를 분석합니다

가장 많이 등장한 Keyword 1위: 한유총 / 등장 횟수 : 331
가장 많이 등장한 Keyword 2위: 개학연기 / 등장 횟수 : 243
가장 많이 등장한 Keyword 3위: 유치원 / 등장 횟수 : 234
가장 많이 등장한 Keyword 4위: 개학 / 등장 횟수 : 166
가장 많이 등장한 Keyword 5위: 연기 / 등장 횟수 : 135
가장 많이 등장한 Keyword 6위: 미세먼지 / 등장 횟수 : 132
가장 많이 등장한 Keyword 7위: 사립유치원 / 등장 횟수 : 86
가장 많이 등장한 Keyword 8위: 기자회견 / 등장 횟수 : 84
가장 많이 등장한 Keyword 9위: 교육감 / 등장 횟수 : 66
가장 많이 등장한 Keyword 10위: 수도권 / 등장 횟수 : 65
가장 많이 등장한 Keyword 11위: 교육부 / 등장 횟수 : 55
가장 많이 등장한 Keyword 12위: 버닝썬 / 등장 횟수 : 52
가장 많이 등장한 Keyword 13위: 선장 / 등장 횟수 : 50
가장 많이 등장한 Keyword 14위: 서울시 / 등장 횟수 : 49
가장 많이 등장한 Keyword 15위: 광안대교 / 등장 횟수 : 47
가장 많이 등장한 Keyword 16위: 전국 / 등장 횟수 : 42
가장 많이 등장한 Keyword 17위: 충돌 / 등장 횟수 : 41
가장 많이 등장한 Keyword 18위: 광주 / 등장 횟수 : 40
가장 많이 등장한 Keyword 19위: 4일 / 등장 횟수 : 40
가장 많이 등장한 Keyword 20위: 경찰 / 등장 횟수 : 40


politics를 분석합니다

가장 많이 등장한 Keyword 1위: 대통령 / 등장 횟수 : 199
가장 많이 등장한 Keyword 2위: 독립유공자 / 등장 횟수 : 136
가장 많이 등장한 Keyword 3위: 황교안 / 등장 횟수 : 101
가장 많이 등장한 Keyword 4위: 해외 / 등장 횟수 : 100
가장 많이 등장한

# TOP10 KEYWORD를 토대로 DataFrame 생성
#   
- 여덟 분야 -> Keyword 10개 -> Keyword 별로 뉴스 제목, URI, 내용 크롤링
#   

In [26]:
# 뉴스회사 크롤링

def news_company_crawl(url):
    html = requests.get(url)
    soup = BeautifulSoup(html.text, 'html.parser')
    soup = str(soup)

    company_first_index = soup.find('property="article:txid"/>\n<meta content')
    company_last_index = soup.find('name="article:media_name')

    comapany = soup[company_first_index+41:company_last_index-2]
    
    return company

In [27]:
# 뉴스 기사 크롤링

def crawl_news_content(url):
    # 줄 별로 크롤링되니 기사들을 모두 합칠 string
    content = ''
    
    # url을 지정
    html = requests.get(url)
    soup = BeautifulSoup(html.text, 'html.parser')
    contents = soup.find_all('p')
    
    # 맨마지막 줄은 기자 이메일이니 제외
    for line in contents[:-2]:
        line = line.text
        # 한줄 한줄 모두 더해줍니다
        content += line
    
    # 번역이 포함되어있으면 감정분석에 시간이 오래걸리므로 번역을 제외해줍니다
    if('kakao i' in content):
        # 번역이 있는 부분을 찾습니다
        translate_index = content.find('kakao i')
        # 번역이 있는 index를 찾아서 제외
        content = content[translate_index+7:]
        
    
    # 언론사 크롤링
    soup = str(soup)
    company_first_index = soup.find('property="article:txid"/>\n<meta content')
    company_last_index = soup.find('name="article:media_name')

    comapany = soup[company_first_index+41:company_last_index-2]
    
    time.sleep(0.1)
    
    return content, comapany

In [45]:
topic_index = 0
keyword_title_list = []
total_url_list = []

# 키워드별로 url을 추출
for topic in topics:
    
    topic_url = []
    
    for keyword in total_keyword_ranking10[topic_index]:
        # 기사 카운트
        
        keyword_url = []
        title_list = []
        
        for i in range(len(title_address_list)-1):
            
            
            
            if keyword in title_address_list[i]:
                
                # 제목에 'http'가 들어가면 패스합니다
                if('http' in title_address_list[i]):
                    continue
                
                keyword_url.append(title_address_list[i+1])
                title_list.append(title_address_list[i])
                
        topic_url.append(keyword_url)
        keyword_title_list.append(title_list)
    topic_index+=1
    
    total_url_list.append(topic_url)

In [29]:
# 기사내용 크롤링

keyword_content_list = []
start_time = time.time()

for total in total_url_list:
    
    # topic
    for url_list in total:
        
        content_list = []
        
        news_info = Parallel(n_jobs=-1)(delayed(crawl_news_content)(url) for url in url_list)
        
        for info in news_info:
            
            content_list.append(info[0])
            
        keyword_content_list.append(content_list)
            
print("걸린시간 : {}분".format(round((time.time() - start_time)/60, 1)))

걸린시간 : 8.8분


In [30]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from joblib import Parallel, delayed

def crawl_daum_comments(url):
    
    chrome_options = webdriver.ChromeOptions()
    prefs = {"profile.managed_default_content_settings.images": 2}
    chrome_options.add_experimental_option("prefs", prefs)
    driver = webdriver.Chrome('/home/apostcto/ITDA/chromedriver', chrome_options=chrome_options)
    driver.get(url)

    
    try:
        elements = driver.find_element_by_class_name('alex_more')

    # 마지막 댓글까지 찾기위해 '더보기' 버튼 클릭
    # 100번이상 클릭하면 문제가 있단 뜻이니 50번넘어가면 break
        count = 0
    
        while(elements):
            try :
                elements.click()
                time.sleep(0.1)
                count+=1
            
                if (count>100):
                    break
                
            except :
                    break
                
    # 더보기 버튼이 없으면 현재 페이지에서 댓글 수집 
    except : 
        pass

    # 최대한 확장된 페이지에서 댓글 요소를 찾습니다
   
    comment_lists = []
    

    # 최대한 확장된 페이지에서 댓글 요소를 찾습니다
    try:
        comment_elements = driver.find_element_by_css_selector('.cmt_news').text
        count_elements = driver.find_element_by_css_selector('.cmt_news .alex_single .cmt_count').text
        count_elements = int(count_elements[3:])
    
    
        while len(comment_elements) > 30:
        # 댓글을 검색
            comment_first_index = comment_elements.find('시간전')
            comment_last_index = comment_elements.find('답글')
            comment = comment_elements[comment_first_index+4:comment_last_index-1].replace('\n','')
            
            if('댓글로그인' in comment):
                comment = ''
            
            comment_lists.append(comment)
        
        # 더이상 댓글이 없으면 break
            if('새로고침' in comment):
                break
    
        # 다음 댓글을 찾기위해 댓글 요소 slicing
            comment_elements = comment_elements[comment_last_index+3:]
        
        return count_elements, comment_lists
    
    except :
        return 0, []

In [34]:
total_keyword_count = []
total_keyword_comments_count = []
keyword_comments_list = []

import time

next = 0

start_time = time.time()

for total in total_url_list:
    
    # topic
    for url_list in total:
        
        keyword_count = len(url_list)
        comments_count = 0
        comments_list = []
        comments_info = Parallel(n_jobs=-1)(delayed(crawl_daum_comments)(url) for url in url_list)
        
        for p in comments_info:
            
            comments_count+=p[0]
            comments_list.append(p[1])
        
        
        total_keyword_count.append(keyword_count)
        total_keyword_comments_count.append(comments_count)
        keyword_comments_list.append(comments_list)
        #total_keyword_comments_list.append(keyword_comments_list)
        
        
        print('{}번째 topic의 검색횟수는 : {}'.format(next+1, total_keyword_count[next]))
        print('{}번째 topic의 총 댓글수는 : {}'.format(next+1, total_keyword_comments_count[next]))
        print('\n')
        next += 1
        
        print("걸린시간 : {}분".format(round((time.time() - start_time)/60, 1)))
        print('\n')
        time.sleep(1)
        
    if(len(total_keyword_count) > 80):
        break

1번째 topic의 검색횟수는 : 473
1번째 topic의 총 댓글수는 : 29668


걸린시간 : 5.5분


2번째 topic의 검색횟수는 : 318
2번째 topic의 총 댓글수는 : 15960


걸린시간 : 9.1분


3번째 topic의 검색횟수는 : 397
3번째 topic의 총 댓글수는 : 17412


걸린시간 : 13.6분


4번째 topic의 검색횟수는 : 511
4번째 topic의 총 댓글수는 : 24456


걸린시간 : 18.7분


5번째 topic의 검색횟수는 : 513
5번째 topic의 총 댓글수는 : 26742


걸린시간 : 24.2분


6번째 topic의 검색횟수는 : 326
6번째 topic의 총 댓글수는 : 5283


걸린시간 : 27.4분


7번째 topic의 검색횟수는 : 101
7번째 topic의 총 댓글수는 : 860


걸린시간 : 28.4분


8번째 topic의 검색횟수는 : 110
8번째 topic의 총 댓글수는 : 668


걸린시간 : 29.6분


9번째 topic의 검색횟수는 : 167
9번째 topic의 총 댓글수는 : 2173


걸린시간 : 31.2분


10번째 topic의 검색횟수는 : 152
10번째 topic의 총 댓글수는 : 2162


걸린시간 : 33.2분


11번째 topic의 검색횟수는 : 335
11번째 topic의 총 댓글수는 : 1594


걸린시간 : 36.5분


12번째 topic의 검색횟수는 : 158
12번째 topic의 총 댓글수는 : 180


걸린시간 : 38.0분


13번째 topic의 검색횟수는 : 141
13번째 topic의 총 댓글수는 : 3942


걸린시간 : 39.9분


14번째 topic의 검색횟수는 : 144
14번째 topic의 총 댓글수는 : 34


걸린시간 : 41.8분


15번째 topic의 검색횟수는 : 160
15번째 topic의 총 댓글수는 : 168


걸린시간 : 43.2분


16번째 topic의 검색횟수

In [38]:
for i in range(80):
    keyword_comments_list[i] = sum(keyword_comments_list[i], [])
    keyword_comments_list[i] = [x for x in keyword_comments_list[i] if x != '']

In [62]:
# 댓글 전처리

replace_comment_list = []

for i in range(80):
    temp = str(keyword_comments_list[i])
    temp = temp.replace('댓글 찬성하기','').replace('댓글 비추천하기','').replace('찬성하기','').replace('비추천하기','').replace('글', '').replace('[','').replace(']','').replace("'',","").replace("\'",'').replace('\'"','')    
    replace_comment_list.append(temp)

In [67]:
# dataframe 길이를 맞춰주기 위한 형식

keyword_comment_ratio = []

for i in range(len(total_keyword_count)):
    try:
        ratio = total_keyword_comments_count[i] / total_keyword_count[i]
        keyword_comment_ratio.append(ratio)
    except:
        keyword_comment_ratio.append(0)
        
topics_80 = []

for topic in topics:
    for i in range(10):
        topics_80.append(topic)
        
keyword_80 = sum(total_keyword_ranking10, [])
company_list = ['Daum' for _ in range(80)]

In [68]:
import pandas as pd

News_dataframe = pd.DataFrame({'Topic':topics_80,
                                'Keyword' : keyword_80,
                                'Company' : company_list,
                                'Title' : keyword_title_list,
                                'Contents' : keyword_content_list,
                                'Comments' : replace_comment_list,
                                'KC' : total_keyword_count,
                                'KCC' : total_keyword_comments_count,
                                'KCR' : keyword_comment_ratio})

News_dataframe

,Topic,Keyword,Company,Title,Contents,Comments,KC,KCC,KCR
0,society,한유총,Daum,"[한유총 광주지회 개학 연기 철회4일 정상 운영, 한유총 광주지회 개학 연기 방침 ...",[-“교육당국이 대화 약속” 교육청 “약속 안해”…갈등 불씨 [헤럴드경제=모바일섹션...,"겁은 나는군..첨부터 그랬으면 얼마나 조켓어요~ㅋ, 교육분 유충들과 대화하지마라개강...",473,29668,62.723044
1,society,개학연기,Daum,"[유치원 개학연기광주 철회 경기인천도 참여 숫자 감소, 경상남도 유치원 개학연기 불...","[한유총 광주지회가 3일 밤 ‘유치원 개학연기 투쟁’을 철회한다고 발표하고, 경기교...",7993127cfo01122019.03.03.14:13저 세금 도둑놈들한유적폐충들...,318,15960,50.188679
2,society,유치원,Daum,"[유치원 개학연기광주 철회 경기인천도 참여 숫자 감소, 유치원 개학 연기 정책이 왜...","[한유총 광주지회가 3일 밤 ‘유치원 개학연기 투쟁’을 철회한다고 발표하고, 경기교...","박근혜때는 신년기자 회견듀 재난 문자로 오더만 그때 기사화 했었나?, 찌라시들에 특...",397,17412,43.858942
3,society,개학,Daum,"[한유총 광주지회 개학 연기 철회4일 정상 운영, 유치원 개학연기광주 철회 경기인천...",[-“교육당국이 대화 약속” 교육청 “약속 안해”…갈등 불씨 [헤럴드경제=모바일섹션...,"박근혜때는 신년기자 회견듀 재난 문자로 오더만 그때 기사화 했었나?, 찌라시들에 특...",511,24456,47.859100
4,society,연기,Daum,"[한유총 광주지회 개학 연기 철회4일 정상 운영, 유치원 개학연기광주 철회 경기인천...",[-“교육당국이 대화 약속” 교육청 “약속 안해”…갈등 불씨 [헤럴드경제=모바일섹션...,"박근혜때는 신년기자 회견듀 재난 문자로 오더만 그때 기사화 했었나?, 찌라시들에 특...",513,26742,52.128655
5,society,미세먼지,Daum,"[내일 9개 시도 미세먼지 저감조치수도권은 나흘 연속, 서울 오후 10시 미세먼지주...",[숨쉬기조차 힘들게 하는 고농도 미세먼지로 수도권을 포함한 일부 지역에 나흘 연속 ...,내일모레 이틀동안 모든 경유차 운행 전면 금지시켜보자.미세먼지가 정말로 획기적으로 ...,326,5283,16.205521
6,society,사립유치원,Daum,"[내일 모든 사립유치원에 공무원 출동해 개학여부 확인, 경남도 공공기관 출퇴근 시간...",[정부가 한국유치원총연합회(한유총)의 '유치원 개학연기 투쟁'에 대응하기 위해 내일...,병설유치원을 증설하고 질과 수준도 높여 사립유치원이 학부모를 쥐락펴락 하지못하게 해...,101,860,8.514851
7,society,기자회견,Daum,"[한유총 관련 수도권 교육감 공동 기자회견 한유총 설립 취소 절차 밟겠다, 한유총 ...",[한국유치원총연합회(이하 한유총)의 무기한 개학 연기 사태에 대해 수도권 3개 시·...,180아메리카노2019.03.03.08:142월 28일까지 원비 입금시키라고 해놓고...,110,668,6.072727
8,society,교육감,Daum,"[ 한유총-수도권 교육감 강대강개학 연기 VS 강행시 설립허가 취소, 한유총 유치원...",[한국유치원총연합회는 3일 무기한 개학 연기 투쟁을 하는 것은 물론 집단폐원 투쟁까...,21광화문바위2019.03.03.13:19이런 한유총은 사회 악. 다 허가 취...,167,2173,13.011976
9,society,수도권,Daum,"[내일 9개 시도 미세먼지 저감조치수도권은 나흘 연속, 내일 9개 시도 미세먼지 저...",[숨쉬기조차 힘들게 하는 고농도 미세먼지로 수도권을 포함한 일부 지역에 나흘 연속 ...,내일모레 이틀동안 모든 경유차 운행 전면 금지시켜보자.미세먼지가 정말로 획기적으로 ...,152,2162,14.223684


In [69]:
News_dataframe.to_csv('daum_news_dataframe', index=False)